# Project 2: Regression and Classification

We will start this project by importing the dataset and cleaning the data as we did in the previous project.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

import sklearn.linear_model as lm
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn import linear_model, model_selection, tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#import torch
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('listings.csv')

In [ ]:
df=df.drop(columns=['neighbourhood_group','license','id','host_id','host_name','calculated_host_listings_count','last_review', 'reviews_per_month', 'number_of_reviews_ltm'])    #drop non-desired columns

In [ ]:
#extract the rating and convert to numeric
rating_format1 = df['name'].str.extract(r'(\d+\.\d+)')  #rating format "x.x"
rating_format2 = df['name'].str.extract(r'(\d+\.\d+)\/')  #rating format "x.x/"
df['rating'] = pd.to_numeric(rating_format1.fillna(rating_format2)[0], errors='coerce')

#extract the num of bed,bedrooms, baths
df['bedroom'] = df['name'].str.extract(r'(\d+)\s+bedroom')  # Extracts the number of bedrooms
df['bed'] = df['name'].str.extract(r'(\d+)\s+bed')  # Extracts the number of beds
df['bath'] = df['name'].str.extract(r'(\d+)\s+bath')  # Extracts the number of baths

#convert the columns to the appropriate data types (int - since it's num of bed,bedrooms, baths)
df['bedroom'] = pd.to_numeric(df['bedroom'], errors='coerce')
df['bed'] = pd.to_numeric(df['bed'], errors='coerce')
df['bath'] = pd.to_numeric(df['bath'], errors='coerce')

#drop 'name' column
df.drop(columns=['name'], inplace=True)

# drop 'bed' column
df = df.drop(columns='bed')

In [ ]:
#If we're missing values in one of these 9 filtered columns, we remove that row..
if df['price'].isna().any():
    print(f"Column has missing values.")
else:
    print(f"Column has no missing values.")

In [ ]:
df = df.fillna(0) # swap NaN for 0

In [ ]:
# filter outliers in price and minimum nights
df = df[(df['price'] >= df['price'].quantile(0.01)) & (df['price'] <= df['price'].quantile(0.99))]

df = df[(df['minimum_nights'] <= df['minimum_nights'].quantile(0.99))]

In [ ]:
# perform log transformation to normalize price and minimum nights since it was right skewed
df['price'] = np.log(df['price'])
df['minimum_nights'] = np.log(df['minimum_nights'])

In [ ]:
# standardize price
df['price'] = (df['price'] - df['price'].mean()) / df['price'].std()

In [ ]:
# normalize number of reviews
df = df[(df['number_of_reviews'] > 0)]
df['number_of_reviews'] = np.log(df['number_of_reviews'])

In [ ]:
# normalize bed
df = df[(df['bedroom'] > 0)]
df['bedroom'] = np.log(df['bedroom'])

In [ ]:
# normalize bath using cubic root
df['bath'] = np.cbrt(df['bath'])

In [ ]:
# standardize the 'latitude' and 'longitude' columns
df['latitude'] = (df['latitude'] - df['latitude'].mean()) / df['latitude'].std()
df['longitude'] = (df['longitude'] - df['longitude'].mean()) / df['longitude'].std()

# binzatize the 'rating' column with 3 as the treshold
df['rating'] = df['rating'].apply(lambda x: 1 if x > 3 else 0)

# standardize bedroom and bath
df['bedroom'] = (df['bedroom'] - df['bedroom'].mean()) / df['bedroom'].std()
df['bath'] = (df['bath'] - df['bath'].mean()) / df['bath'].std()

# binarize minimum nights with the median as the threshold
df['minimum_nights'] = df['minimum_nights'].apply(lambda x: 1 if x > df['minimum_nights'].median() else 0)

# binarize number of reviews with the median as the threshold
df['number_of_reviews'] = df['number_of_reviews'].apply(lambda x: 1 if x > df['number_of_reviews'].median() else 0)

# binarize availability_365 with the median as the threshold
df['availability_365'] = df['availability_365'].apply(lambda x: 1 if x > df['availability_365'].median() else 0)

# perform one-out-of-k encoding on the 'room_type'
df = pd.get_dummies(df, columns=['room_type'], dtype=int)

In [ ]:
df.describe().applymap("{0:.2f}".format)

In [ ]:
df = df.drop(columns='neighbourhood')

In [ ]:
df.head()

### Regression part a)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
criterion_variables = df[['price', 'latitude', 'longitude', 'bedroom', 'bath']]

X = df.drop('price', axis=1)  # Features
y = df['price']  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Categorical and numerical column names
categorical_cols = ['room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room']
binary_cols = ['number_of_reviews', 'availability_365', 'minimum_nights', 'rating']
numeric_cols = ['latitude', 'longitude', 'bedroom', 'bath']

In [ ]:
# Define range of lambda values to test
lambdas = np.logspace(-4, 0.2, 50)

all_errors = {}

generalization_errors = []

# Define number of folds for KFold cross-validation
n_folds = 10

# Create KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_number = 0
fold_error_list = []
# Loop through each fold
for train_index, test_index in kf.split(X):
    fold_number += 1
    print(f"Fold {fold_number} of {n_folds}...")
    # Split data into training and testing sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize list to store error for each fold
    error_list = []

    for lmbda in lambdas:
        # Create Ridge regression object
        model = Ridge(lmbda)

        # Fit the model on the training data for this fold
        model.fit(X_train, y_train)
        
        # Make predictions on the testing data for this fold
        predictions = model.predict(X_test)
        
        # Calculate the mean squared error for this fold
        mse = mean_squared_error(y_test, predictions)
        
        # Append the error to the error list
        error_list.append(mse)

    fold_error_list.append(error_list)
    

    # Calculate the generalization error for this fold
    generalization_error = np.mean(error_list)

    err_dict = {'best_lambda': lambdas[error_list.index(min(error_list))], 'error': min(error_list)}

    all_errors[fold_number] = err_dict


    # Append the generalization error to the list
    generalization_errors.append(generalization_error)


In [ ]:
for error in all_errors:
    print(f"Fold {error}: {all_errors[error]}")

In [ ]:
best_fold = 4

print(f"Minimum Generalization Error: {min(generalization_errors)}")
print(f"Lambda value with minimum generalization error: {lambdas[generalization_errors.index(min(generalization_errors))]}")

#Average generalisation error vs regularisation parameter (lambda)
plt.figure(figsize=(10, 6))
plt.semilogx(lambdas, fold_error_list[best_fold-1], marker='o')
plt.xlabel('λ (Regularization Parameter)')
plt.ylabel('Average Generalization Error')
plt.title('Generalization Error as a Function of λ')
plt.grid(True)
plt.show()

The generalization error curve remains low for a range of lambda values and then increases at the end without exhibiting a clear U-shaped pattern. That suggests that the model is not very sensitive to changes in the regularization parameter (lambda) over the specified range. A lack of a U-shaped curve indicates that the model is robust, meaning that it doesn't seem to overfit (high variance) or underfit (high bias) across the range of lambda values tested. The most optimal lambda value is hard to distiguish, since the one that minimizes the generalization error, but in this case, it may not be distinctly observable.

In [ ]:
feature_names = df.columns
lambda_values = np.logspace(-1, 6, 100)
coefficients = []

#fitting Ridge regression models for different lambda values and storing coefficients
for alpha in lambda_values:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    coefficients.append(ridge.coef_)

#coefficient profile plot
plt.figure(figsize=(8, 6))
plt.plot(lambda_values, coefficients)
plt.xscale('log')
plt.xlabel('Lambda (Regularization Strength)')
plt.ylabel('Coefficients')
plt.title('Ridge Regression Coefficient Profile')
plt.legend(feature_names, loc='center left', bbox_to_anchor=(1, 0.5))

plt.axis('tight')
plt.show()

The ridge regression "penalizes" the variable coefficients, such that those that are the least effective, "shrink" the fastest. In general, as alpha increases, the pentalty for large coefficients also increases. This decreases the absolute values of the coefficients. If certain coefficients flatten out sooner than others, it represents reaching the regularisation state. If the coefficient values remain large, it means that it may be a significant features when making predictions. If certain coefficients flatten out later than others, it means that they may be insignificant features when making predictions. 

Imagine you have a budget allocated and each coefficient can take some to play a role in the estimation. Naturally those who are more important will take more of the budget. As you increase the lambda, you are decreasing the budget, i.e. penalizing more.

### Regression Part b)

In this section, we will compare three models: the regularized linear regression model from the previous section, an artificial neural network and a baseline

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize list to store generalization errors for each lambda value
generalization_errors = []

# Loop through each fold
mean_y_train = np.mean(y_train)

fold = 0

# compute baseline error for 10 outer folds
for train_index, test_index in kf.split(X):
    fold += 1
    # Split data into training and testing sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize list to store error for each fold
    error_list = []

    # Compute the mean of y_train
    y_mean = np.mean(y_train)

    # Compute the baseline MSE for this fold
    baseline_mse = mean_squared_error(y_test, np.full(len(y_test), y_mean))

    print(f"Baseline MSE for fold {fold}: {baseline_mse}")

# Calculate the average generalization error
average_generalization_error = np.mean(generalization_errors)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

generalization_errors = []

# Define number of folds for KFold cross-validation
n_folds = 10

# Create KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

fold_number = 0
fold_error_list = []

param_distributions = {
            'hidden_layer_sizes': [(1,), (10,), (25,), (40,), (50,)]  # Testing different sizes for a single hidden layer
        }

errors_by_fold = []

# Loop through each fold
for train_index, test_index in kf.split(X_train):
    
    fold_number += 1

    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]

    # Create MLP regression object
    ann = MLPRegressor(hidden_layer_sizes=param_distributions, activation='relu', solver='adam', max_iter=10000, random_state=42)

    search = RandomizedSearchCV(estimator=ann, param_distributions=param_distributions, cv=10, scoring='neg_mean_squared_error')    
    search.fit(X_train, y_train)

    # Get the best estimator from the search
    best_ann = search.best_estimator_
    hidden_layer_size = best_ann.get_params()['hidden_layer_sizes']

    # Fit the best estimator on the training data
    best_ann.fit(X_train, y_train)

    # Make predictions on the testing data
    predictions = best_ann.predict(X_fold_test)

    # Calculate the mean squared error
    mse = mean_squared_error(y_fold_test, predictions)

    print(f"Current fold: {fold_number},  Optimal hidden layer size: {hidden_layer_size}, Error: {mse:.4f}")

In [ ]:
from scipy import stats
from scipy.stats import ttest_rel

# List of MSEs for each fold from ANN
mse_ann = [
    0.4416,
    0.3905,
    0.4795,
    0.4795,
    0.4709,
    0.4575,
    0.4304,
    0.4601,
    0.4143,
    0.4377
]  

# List of MSEs for each fold from Linear Regression
mse_linear = [
    0.5462907953225147,
    0.546975763341854,
    0.5752950136028192,
    0.500884931457507,
    0.6080982778664757,
    0.5343998159373244,
    0.5372433882424416,
    0.5131378558592264,
    0.5268944003529467,
    0.5715907993105115
]  

# List of MSEs for each fold from Baseline
mse_baseline = [
    1.0160014312259356,
    0.9565232234464234,
    1.00042283226036,
    0.9585350644543394,
    1.0328813178777085,
    1.0234779533815574,
    1.030870624784249,
    0.9676963853623705,
    1.0058567962898126,
    1.0136289464187866
]  

# Function to calculate the confidence interval
def compute_confidence_interval(data1, data2, confidence=0.95):
    diff = np.array(data1) - np.array(data2)
    mean_diff = np.mean(diff)
    sem = stats.sem(diff)
    dof = len(diff) - 1
    t_crit = np.abs(stats.t.ppf((1 - confidence) / 2, dof))
    margin_of_error = t_crit * sem
    lower_bound = mean_diff - margin_of_error
    upper_bound = mean_diff + margin_of_error
    return (lower_bound, upper_bound)

# Performing paired t-tests and calculating confidence intervals
t_statistic_ann_vs_linear, p_value_ann_vs_linear = ttest_rel(mse_ann, mse_linear)
ci_ann_vs_linear = compute_confidence_interval(mse_ann, mse_linear)

t_statistic_ann_vs_baseline, p_value_ann_vs_baseline = ttest_rel(mse_ann, mse_baseline)
ci_ann_vs_baseline = compute_confidence_interval(mse_ann, mse_baseline)

t_statistic_linear_vs_baseline, p_value_linear_vs_baseline = ttest_rel(mse_linear, mse_baseline)
ci_linear_vs_baseline = compute_confidence_interval(mse_linear, mse_baseline)

# Printing the results
print(f"ANN vs. Linear Regression: t-statistic = {t_statistic_ann_vs_linear}, p-value = {p_value_ann_vs_linear}, CI = {ci_ann_vs_linear}")
print(f"ANN vs. Baseline: t-statistic = {t_statistic_ann_vs_baseline}, p-value = {p_value_ann_vs_baseline}, CI = {ci_ann_vs_baseline}")
print(f"Linear Regression vs. Baseline: t-statistic = {t_statistic_linear_vs_baseline}, p-value = {p_value_linear_vs_baseline}, CI = {ci_linear_vs_baseline}")

## Classification 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

price_quantiles = df['price'].quantile([0.33, 0.66])
low_threshold = price_quantiles[0.33]
high_threshold = price_quantiles[0.66]

# Function to categorize prices
def categorize_price(price):
    if price <= low_threshold:
        return 'low'
    elif price <= high_threshold:
        return 'medium'
    else:
        return 'high'

# Apply the function to create a new categorical variable
df['price_category'] = df['price'].apply(categorize_price)

Select lambda value for classification, optimal....

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
import pandas as pd

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define range of lambda values to test
lambda_values = np.logspace(-4, 4, 100)
C_values = 1 / lambda_values  # Convert lambda values to C values for Logistic Regression

all_errors = {}

# Initialize list to store generalization errors for each C value
generalization_errors = []

# Define number of folds for KFold cross-validation
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)


fold_number = 0
fold_error_list = []

# Loop through each fold
for train_index, test_index in kf.split(X_train):
    # Split data into training and testing sets for this fold
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Initialize list to store error for each fold
    error_list = []

    # Loop through each C value
    for c in C_values:
        # Create Multinomial Logistic Regression model object with given C value
        model = LogisticRegression(C=c, max_iter=1000, solver='lbfgs', random_state=42)
        
        # Fit the model on the training data for this fold
        model.fit(X_fold_train, y_fold_train)
        
        # Make predictions on the testing data for this fold
        fold_pred = model.predict(X_fold_test)
        
        # Calculate the error (1 - accuracy) for this fold
        fold_error = 1 - accuracy_score(y_fold_test, fold_pred)
        
        # Append the error to the error list
        error_list.append(fold_error)

    fold_error_list.append(error_list)

    # Calculate the average error across all folds
    generalization_error = np.mean(error_list)

    err_dict = {'best_lambda': (lambda_values[error_list.index(min(error_list))]), 'error': min(error_list)}

    all_errors[fold_number] = err_dict

    # Append the generalization error to the list
    generalization_errors.append(generalization_error)

    fold_number += 1

In [ ]:
# Initialize matrix to store errors for each C value across all folds
errors_matrix = np.zeros((len(C_values), n_folds))

# Loop through each fold
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train)):
    # Split data into training and testing sets for this fold
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Loop through each C value
    for c_idx, c in enumerate(C_values):
        # Create Logistic Regression model object with given C value
        model = LogisticRegression(C=c, max_iter=1000, solver='lbfgs', random_state=42)
        
        # Fit the model on the training data for this fold
        model.fit(X_fold_train, y_fold_train)
        
        # Make predictions on the testing data for this fold
        fold_pred = model.predict(X_fold_test)
        
        # Calculate the error (1 - accuracy) for this fold
        fold_error = 1 - accuracy_score(y_fold_test, fold_pred)
        
        # Store the error in the matrix
        errors_matrix[c_idx, fold_idx] = fold_error

# Calculate the average error across all folds for each C value
generalization_errors = np.mean(errors_matrix, axis=1)

# Plotting the generalization error for different C values
plt.figure(figsize=(10, 6))
plt.plot(C_values, generalization_errors, marker='o')
plt.xscale('log')
plt.xlabel('C value (Inverse of Lambda)')
plt.ylabel('Average Generalization Error')
plt.title('Generalization Error vs. C value in Logistic Regression')
plt.show()

In [ ]:
for error in all_errors:
    print(f"Fold {error}: {all_errors[error]}")

In [ ]:
best_fold = 9

print(f"Minimum Generalization Error: {min(generalization_errors)}")
print(f"Lambda value with minimum generalization error: {(lambda_values[generalization_errors.index(min(generalization_errors))])}")

#Average generalisation error vs regularisation parameter (lambda)
plt.figure(figsize=(10, 6))
plt.semilogx(lambda_values, fold_error_list[best_fold-1], marker='o')
plt.xlabel('λ (Regularization Parameter)')
plt.ylabel('Average Generalization Error')
plt.title('Generalization Error as a Function of λ')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Lambda values for Logistic Regression
lambda_values = np.logspace(-1, 1, 50)
C_values = 1 / lambda_values

# Define outer and inner CV splits
outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=10, shuffle=True, random_state=43)

results = []

fold_number = 0
for train_idx, test_idx in outer_cv.split(X, y):
    fold_number += 1
    print(f"Fold {fold_number} running.")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Baseline model: predicting the most common class
    most_common_class = y_train.value_counts().idxmax()
    y_pred_base = [most_common_class] * len(y_test)
    error_rate_base = 1 - accuracy_score(y_test, y_pred_base)
    
    # Logistic Regression with CV for hyperparameter tuning
    best_acc_logreg = 0
    best_c = None
    for c in C_values:
        logreg = LogisticRegression(C=c, max_iter=1000, multi_class='multinomial', solver='lbfgs')
        inner_accs = []
        for inner_train_idx, inner_val_idx in inner_cv.split(X_train, y_train):
            X_inner_train, X_val = X_train.iloc[inner_train_idx], X_train.iloc[inner_val_idx]
            y_inner_train, y_val = y_train.iloc[inner_train_idx], y_train.iloc[inner_val_idx]
            
            logreg.fit(X_inner_train, y_inner_train)
            y_pred = logreg.predict(X_val)
            acc = accuracy_score(y_val, y_pred)
            inner_accs.append(acc)
        
        avg_acc = np.mean(inner_accs)
        if avg_acc > best_acc_logreg:
            best_acc_logreg = avg_acc
            best_c = c
    
    # ANN with Randomized Search for hyperparameter tuning
    ann = MLPClassifier(max_iter=1000, random_state=42)
    param_distributions = {
        'hidden_layer_sizes': [(1,), (15,), (30,), (50,)]  # Testing different sizes for a single hidden layer
    }
    random_search = RandomizedSearchCV(ann, param_distributions, n_iter=4, cv=inner_cv, scoring='accuracy', random_state=44, n_jobs=-1)
    random_search.fit(X_train, y_train)
    
    best_ann = random_search.best_estimator_
    optimal_hidden_layer_size = best_ann.get_params()['hidden_layer_sizes']
    # Error rate for the best models on the test set
    logreg_best = LogisticRegression(C=best_c, max_iter=1000, multi_class='multinomial', solver='lbfgs')
    logreg_best.fit(X_train, y_train)
    y_pred_logreg = logreg_best.predict(X_test)
    
    y_pred_ann = best_ann.predict(X_test)
    
    error_rate_logreg = 1 - accuracy_score(y_test, y_pred_logreg)
    error_rate_ann = 1 - accuracy_score(y_test, y_pred_ann)
    
    results.append((error_rate_base, error_rate_logreg, error_rate_ann, best_c, optimal_hidden_layer_size))

for result in results:
    print(f"Baseline Error: {result[0]:.4f}, LogReg Error (C={result[3]}): {result[1]:.4f}, "
          f"ANN Error (hidden_layer_sizes={result[4]}): {result[2]:.4f}")


In [ ]:
import pandas as pd

columns = ['Outer fold', 'Logistic regression', 'ANN', 'Baseline', 'λ (Logistic Regression)', 'α (ANN)']
data = []
for i, (base, logreg, ann, c, alpha) in enumerate(results, 1):
    data.append([i, logreg, ann, base, 1/c, alpha])

df = pd.DataFrame(data, columns=columns)
print(df)

In [ ]:
from sklearn.metrics import accuracy_score
from statsmodels.stats.contingency_tables import mcnemar

In [ ]:
# Function to create a contingency table
def create_contingency_table(predictions1, predictions2):
    return np.array([
        sum((predictions1 == y_test) & (predictions2 == y_test)),
        sum((predictions1 == y_test) & (predictions2 != y_test)),
        sum((predictions1 != y_test) & (predictions2 == y_test)),
        sum((predictions1 != y_test) & (predictions2 != y_test))
    ]).reshape(2, 2)

# Creating contingency tables
contingency_table_1 = create_contingency_table(y_pred_logreg, y_pred_ann)
contingency_table_2 = create_contingency_table(y_pred_logreg, y_pred_base)
contingency_table_3 = create_contingency_table(y_pred_ann, y_pred_base)

# Performing McNemar's tests
result_1 = mcnemar(contingency_table_1, exact=False)
result_2 = mcnemar(contingency_table_2, exact=False)
result_3 = mcnemar(contingency_table_3, exact=False)

# Printing the results
print("Logistic Regression vs MLP Classifier:")
print(f"p-value: {result_1.pvalue}")

print("Logistic Regression vs Dummy Classifier:")
print(f"p-value: {result_2.pvalue}")

print("MLP Classifier vs Dummy Classifier:")
print(f"p-value: {result_3.pvalue}")